1. I was about to clean the collected corpus from internetarchive. Then I looked at what others have done and decided to scrape from Khasi newspapers for a larger corpus.

In [1]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_mawphor_articles(base_url, num_pages=5):
    articles = []

    for page in range(1, num_pages + 1):
        url = f"{base_url}/page/{page}/"
        print(f"Scraping: {url}")
        
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', href=True)

        # Filter article URLs (customize this per site)
        article_links = [a['href'] for a in links if '/news/' in a['href']]
        article_links = list(set(article_links))  # Remove duplicates

        for article_url in article_links:
            try:
                article_response = requests.get(article_url)
                article_soup = BeautifulSoup(article_response.text, 'html.parser')
                content_div = article_soup.find('div', class_='post-content')  # Adjust class name
                if content_div:
                    article_text = content_div.get_text(separator=' ', strip=True)
                    articles.append(article_text)
                time.sleep(1)  # be polite
            except Exception as e:
                print(f"Error scraping {article_url}: {e}")

    return articles

# Example use
khasi_articles = scrape_mawphor_articles("https://mawphor.com", num_pages=3)

# Save to file
with open("downloads_newspapers/khasi_corpus.txt", "w", encoding="utf-8") as f:
    for article in khasi_articles:
        f.write(article + "\n")



Scraping: https://mawphor.com/page/1/
Scraping: https://mawphor.com/page/2/
Scraping: https://mawphor.com/page/3/
